In [121]:
import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder

### Read data

In [148]:
df = pd.read_csv('data-related/data.csv')

### Preprocess data

In [149]:
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.rstrip('_')

In [152]:
df

,Подразделение_1,Функциональный_блок,Подразделение_2,Подразделение_3,Подразделение_4,Должность,Роль,Фамилия,Имя,Телефон,Город,Адрес,Почта
0,Центральный офис,NaN,NaN,NaN,NaN,Председатель банка,руководство,Смирнов,Александр,79699822593,Москва,"проспект Мира, 14, 3 этаж, офис 301",NaN
1,Центральный офис,Корпоративный блок,NaN,NaN,NaN,Заместитель председателя банка,руководство,Иванов,Алексей,79227639290,Москва,"проспект Мира, 14, 3 этаж, офис 302",NaN
2,Центральный офис,Корпоративный блок,онлайн-банкинг для бизнеса,NaN,NaN,Владелец продукта,руководство,Кузнецов,Альберт,79182913214,Москва,"проспект Мира, 14, 2 этаж, место 1",NaN
3,Центральный офис,Корпоративный блок,онлайн-банкинг для бизнеса,команда 1,NaN,UX/UI Дизайнер,Дизайнер,Соколов,Александр,79700231854,Москва,"проспект Мира, 14, 2 этаж, место 2",NaN
4,Центральный офис,Корпоративный блок,онлайн-банкинг для бизнеса,команда 1,NaN,UX Дизайнер,Дизайнер,Попов,Алексей,79254156549,Москва,"проспект Мира, 14, 2 этаж, место 3",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Руководитель офиса,руководство,Соловьёва,Акулина,79833706058,Волгоград,улица Татищева 27,NaN
210,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Менеджер по продажам,продажи,Васильева,Алевтина,79681314418,Волгоград,улица Татищева 27,NaN
211,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Ведущий менеджер по продажам,продажи,Зайцева,Александра,79515700252,Волгоград,улица Татищева 27,NaN
212,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Кассир,обслуживание,Павлов,Алексей,79997618652,Волгоград,улица Татищева 27,NaN


### kNN

In [154]:
df_encoded = df_encoded.drop(columns=['Телефон', 'Адрес', 'Почта'])

In [155]:
df_encoded

,Подразделение_1,Функциональный_блок,Подразделение_2,Подразделение_3,Подразделение_4,Должность,Роль,Фамилия,Имя,Город
0,Центральный офис,NaN,NaN,NaN,NaN,Председатель банка,руководство,Смирнов,Александр,Москва
1,Центральный офис,Корпоративный блок,NaN,NaN,NaN,Заместитель председателя банка,руководство,Иванов,Алексей,Москва
2,Центральный офис,Корпоративный блок,онлайн-банкинг для бизнеса,NaN,NaN,Владелец продукта,руководство,Кузнецов,Альберт,Москва
3,Центральный офис,Корпоративный блок,онлайн-банкинг для бизнеса,команда 1,NaN,UX/UI Дизайнер,Дизайнер,Соколов,Александр,Москва
4,Центральный офис,Корпоративный блок,онлайн-банкинг для бизнеса,команда 1,NaN,UX Дизайнер,Дизайнер,Попов,Алексей,Москва
...,...,...,...,...,...,...,...,...,...,...
209,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Руководитель офиса,руководство,Соловьёва,Акулина,Волгоград
210,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Менеджер по продажам,продажи,Васильева,Алевтина,Волгоград
211,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Ведущий менеджер по продажам,продажи,Зайцева,Александра,Волгоград
212,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Кассир,обслуживание,Павлов,Алексей,Волгоград


In [168]:
encoder = OneHotEncoder(handle_unknown='ignore')

In [169]:
encoded_data = encoder.fit_transform(df_encoded.fillna('None')).toarray()

In [170]:
encoded_data

array([[0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [199]:
nn = NearestNeighbors(n_neighbors=10, metric='euclidean')

In [200]:
nn.fit(encoded_data)

NearestNeighbors(metric='euclidean', n_neighbors=10)

In [217]:
input_data = {
    'Подразделение_1': "Центральный офис",
    'Имя': 'Александр',
    'Фамилия': 'Смирнов'
}

In [218]:
input_df = pd.DataFrame([input_data])

In [219]:
for col in df_encoded.columns:
    if col not in input_df.columns:
        input_df[col] = 'None'

In [220]:
input_df = input_df[df_encoded.columns]

In [221]:
encoded_input_data = encoder.transform(input_df).toarray()

In [222]:
distances, indices = nn.kneighbors(encoded_input_data)

In [223]:
print("Индексы ближайших соседей:", indices)

Индексы ближайших соседей: [[  0   1  66  25  31 143  15  27  28  26]]


In [224]:
nearest_neighbors = df.iloc[indices[0]]
nearest_neighbors

,Подразделение_1,Функциональный_блок,Подразделение_2,Подразделение_3,Подразделение_4,Должность,Роль,Фамилия,Имя,Телефон,Город,Адрес,Почта
0,Центральный офис,NaN,NaN,NaN,NaN,Председатель банка,руководство,Смирнов,Александр,79699822593,Москва,"проспект Мира, 14, 3 этаж, офис 301",NaN
1,Центральный офис,Корпоративный блок,NaN,NaN,NaN,Заместитель председателя банка,руководство,Иванов,Алексей,79227639290,Москва,"проспект Мира, 14, 3 этаж, офис 302",NaN
66,Филиал в Оренбургской области,NaN,NaN,NaN,NaN,Управляющий филиала,руководство,Осипова,Анастасия,79565126979,Оренбург,"проспект Ленина 23, 2 этаж, 201",NaN
25,Центральный офис,Корпоративный блок,Управление по работе с филиалами,NaN,NaN,Директор управления,руководство,Комаров,Александр,79667036549,Москва,"проспект Мира, 14, 2 этаж, место 24",NaN
31,Центральный офис,Розничный блок,NaN,NaN,NaN,Заместитель председателя банка,руководство,Ильина,Александра,79014813949,Москва,"проспект Мира, 14, 3 этаж, офис 303",NaN
143,Филиал в Волгоградской области,NaN,NaN,NaN,NaN,Управляющий филиала,руководство,Жданов,Валентин,79057887457,Волгоград,"улица Мира 9, 501",NaN
15,Центральный офис,Корпоративный блок,транзакционные продукты,NaN,NaN,Владелец продукта,руководство,Семёнов,Александр,79382059816,Москва,"проспект Мира, 14, 2 этаж, место 14",NaN
27,Центральный офис,Корпоративный блок,Управление по работе с филиалами,NaN,NaN,Территориальный менеджер,руководство,Киселёв,Альберт,79716028148,Москва,"проспект Мира, 14, 2 этаж, место 26",NaN
28,Центральный офис,Корпоративный блок,Управление по работе с филиалами,NaN,NaN,Территориальный менеджер,руководство,Макаров,Георгий,79563971776,Москва,"проспект Мира, 14, 2 этаж, место 27",NaN
26,Центральный офис,Корпоративный блок,Управление по работе с филиалами,NaN,NaN,Территориальный менеджер,руководство,Орлов,Алексей,79612359514,Москва,"проспект Мира, 14, 2 этаж, место 25",NaN


In [202]:
query = {
    'Подразделение_1': 'Центральный офис',
    'Функциональный_блок': 'Корпоративный блок',
    'Фамилия': 'Иванов',
    'Имя': 'Алексей',
    'Роль': 'руководство'
}

columns = ['Подразделение_1', 'Функциональный_блок', 'Подразделение_2', 'Подразделение_3', 'Подразделение_4', 'Должность',  'Роль', 'Фамилия', 'Имя', 'Город']

In [203]:
df['combined'] = df[columns].astype(str).agg(' '.join, axis=1)

In [204]:
query_df = pd.DataFrame([query], columns=columns)
query_df['combined'] = query_df[columns].astype(str).agg(' '.join, axis=1)

In [205]:
query_df

,Подразделение_1,Функциональный_блок,Подразделение_2,Подразделение_3,Подразделение_4,Должность,Роль,Фамилия,Имя,Город,combined
0,Центральный офис,Корпоративный блок,NaN,NaN,NaN,NaN,руководство,Иванов,Алексей,NaN,Центральный офис Корпоративный блок nan nan na...


In [221]:
combined_df = pd.concat([df[['combined']], query_df[['combined']]], ignore_index=True)[:-1]
combined_df

,combined
0,Центральный офис nan nan nan nan Председатель ...
1,Центральный офис Корпоративный блок nan nan na...
2,Центральный офис Корпоративный блок онлайн-бан...
3,Центральный офис Корпоративный блок онлайн-бан...
4,Центральный офис Корпоративный блок онлайн-бан...
...,...
209,Филиал в Волгоградской области Розничный блок ...
210,Филиал в Волгоградской области Розничный блок ...
211,Филиал в Волгоградской области Розничный блок ...
212,Филиал в Волгоградской области Розничный блок ...


In [222]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(combined_df['combined'])

In [223]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [225]:
sim_scores = list(enumerate(cosine_sim[-1]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

In [226]:
df_indices = [i[0] for i in sim_scores]
required = df_indices[:5]

In [228]:
df.iloc[required]

,Подразделение_1,Функциональный_блок,Подразделение_2,Подразделение_3,Подразделение_4,Должность,Роль,Фамилия,Имя,Телефон,Город,Адрес,Почта,combined
213,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Старший кассир,обслуживание,Семёнов,Альберт,79088534033,Волгоград,улица Татищева 27,NaN,Филиал в Волгоградской области Розничный блок ...
128,Филиал в Оренбургской области,Розничный блок,NaN,NaN,Дополнительный офис 2,Кассир,обслуживание,Горшков,Альберт,79252830363,Оренбург,"улица Черных, 10",NaN,Филиал в Оренбургской области Розничный блок n...
208,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 3,Старший кассир,обслуживание,Волкова,Евгения,79650029360,Волгоград,улица Фрунзе 12,NaN,Филиал в Волгоградской области Розничный блок ...
196,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 1,Старший кассир,обслуживание,Мишина,Анастасия,79129575289,Волгоград,"улица Мира 9, 101",NaN,Филиал в Волгоградской области Розничный блок ...
201,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 2,Старший кассир,обслуживание,Соколова,Ольга,79207231639,Волгоград,улица Куйбышева 29,NaN,Филиал в Волгоградской области Розничный блок ...


In [ ]:
def recommend_rows(df, query, columns):